In [5]:
if 'autoreload' not in get_ipython().extension_manager.loaded:
    %load_ext autoreload 
%autoreload 2

import numpy as np
from os import PathLike
from typing import Union, List

from ray import tune
from ray.air import session
from ray.tune.integration.pytorch_lightning import TuneReportCallback
from ray.tune.schedulers import ASHAScheduler, MedianStoppingRule, PopulationBasedTraining
from ray.tune.search import BasicVariantGenerator
from ray.tune.search.bayesopt import BayesOptSearch 
from ray.tune.search.hyperopt import HyperOptSearch

from pytorch_lightning import Trainer, seed_everything
from pytorch_lightning.loggers import TensorBoardLogger
from torch.utils.data import DataLoader

import eugene as eu

In [6]:
def hyperopt_with_tune(
    config: dict,
    sdata = None,
    target_keys: Union[str, List[str]] = None,
    train_key: str = "train_val",
    epochs: int = 10,
    gpus: int = None,
    num_workers: int = None,
    log_dir: PathLike = None,
    name: str = None,
    version: str = None,
    train_dataset: eu.dl.SeqDataset = None,
    val_dataset: eu.dl.SeqDataset = None,
    train_dataloader: DataLoader = None,
    val_dataloader: DataLoader = None,
    seq_transforms: List[str] = None,
    transform_kwargs: dict = {},
    seed: int = None,
    verbosity = None,
):
    model = eu.models.get_model(config["arch"], config)
    gpus = gpus if gpus is not None else eu.settings.gpus
    num_workers = num_workers if num_workers is not None else eu.settings.dl_num_workers
    log_dir = log_dir if log_dir is not None else eu.settings.logging_dir
    name = name if name is not None else config["arch"]
    seed_everything(seed, workers=True) if seed is not None else seed_everything(eu.settings.seed)
    batch_size = config["batch_size"]
    if train_dataloader is not None:
        assert val_dataloader is not None
    elif train_dataset is not None:
        assert val_dataset is not None
        train_dataloader = DataLoader(
            train_dataset, batch_size=batch_size, num_workers=num_workers
        )
        val_dataloader = DataLoader(
            val_dataset, batch_size=batch_size, num_workers=num_workers
        )
    elif sdata is not None:
        assert target_keys is not None
        targs = sdata.seqs_annot[target_keys].values  
        if len(targs.shape) == 1:
            nan_mask = np.isnan(targs)
        else:
            nan_mask = np.any(np.isnan(targs), axis=1)
        print(f"Dropping {nan_mask.sum()} sequences with NaN targets.")
        sdata = sdata[~nan_mask]
        train_idx = np.where(sdata.seqs_annot[train_key] == True)[0]
        train_dataset = sdata[train_idx].to_dataset(
            target_keys=target_keys,
            seq_transforms=seq_transforms,
            transform_kwargs=transform_kwargs,
        )
        train_dataloader = train_dataset.to_dataloader(
            batch_size=batch_size, num_workers=num_workers, shuffle=True
        )
        val_idx = np.where(sdata.seqs_annot[train_key] == False)[0]
        val_dataset = sdata[val_idx].to_dataset(
            target_keys=target_keys,
            seq_transforms=seq_transforms,
            transform_kwargs=transform_kwargs,
        )
        val_dataloader = val_dataset.to_dataloader(
            batch_size=batch_size,
            num_workers=num_workers,
            shuffle=False,
        )
    else:
        raise ValueError("No data provided to train on.")
    logger = TensorBoardLogger(log_dir, name=name, version=version)
    callbacks = []
    metrics = {"loss": "val_loss"}
    callbacks.append(TuneReportCallback(metrics, on="validation_end"))
    trainer = Trainer(
        max_epochs=epochs,
        gpus=gpus,
        logger=logger,
        progress_bar_refresh_rate=0,
        callbacks=callbacks,
    )
    trainer.fit(model, train_dataloaders=train_dataloader, val_dataloaders=val_dataloader)


In [9]:
scheduler_dict = {
    "ASHAScheduler": ASHAScheduler,
    "MedianStoppingRule": MedianStoppingRule,
    "PopulationBasedTraining": PopulationBasedTraining,
    
}

algo_dict = {
    "BayesOptSearch": BayesOptSearch,
    "HyperOptSearch": HyperOptSearch,
    "BasicVariantGenerator": BasicVariantGenerator,
}

def hyperopt(
    config,
    scheduler="ASHAScheduler",
    algorithm="BasicVariantGenerator",
    sdata = None,
    target_keys: Union[str, List[str]] = None,
    train_key: str = "train_val",
    epochs: int = 10,
    gpus: int = None,
    num_workers: int = None,
    log_dir: PathLike = None,
    name: str = None,
    version: str = None,
    train_dataset: eu.dl.SeqDataset = None,
    val_dataset: eu.dl.SeqDataset = None,
    train_dataloader: DataLoader = None,
    val_dataloader: DataLoader = None,
    seq_transforms: List[str] = None,
    transform_kwargs: dict = {},
    seed: int = None,
    verbosity = None,
    scheduler_kwargs: dict = {},
    algorithm_kwargs: dict = {},
    **kwargs
):
    trainable = tune.with_parameters(
        hyperopt_with_tune,
        sdata=sdata,
        target_keys=target_keys,
        train_key=train_key,
        epochs=epochs,
        gpus=gpus,
        num_workers=num_workers,
        log_dir=log_dir,
        name=name,
        version=version,
        train_dataset=train_dataset,
        val_dataset=val_dataset,
        train_dataloader=train_dataloader,
        val_dataloader=val_dataloader,
        seq_transforms=seq_transforms,
        transform_kwargs=transform_kwargs,
        seed=seed,
        verbosity=verbosity,
        **kwargs
    )
    algo = algo_dict[algorithm](**algorithm_kwargs)
    scheduler = scheduler_dict[scheduler](**scheduler_kwargs)
    scheduler = ASHAScheduler(
       
    )
    analysis = tune.run(
        trainable,
        config=config,
        metric="loss",
        mode="min",
        num_samples=10,
        local_dir=eu.settings.logging_dir,
        keep_checkpoints_num=1,
        checkpoint_score_attr="min-val_loss",
        name=name
    )

    return analysis

In [10]:
sdata = eu.datasets.random1000()
eu.pp.ohe_seqs_sdata(sdata)
eu.pp.reverse_complement_seqs_sdata(sdata)
eu.pp.train_test_split_sdata(sdata)

One-hot encoding sequences:   0%|          | 0/1000 [00:00<?, ?it/s]

SeqData object modified:
	ohe_seqs: None -> 1000 ohe_seqs added
SeqData object modified:
	ohe_rev_seqs: None -> 1000 ohe_rev_seqs added
SeqData object modified:
    seqs_annot:
        + train_val


In [24]:
scheduler_kwargs = dict(
    metric="loss",
    mode="min",
    max_t=10,
    grace_period=1,
    reduction_factor=2
)
algo_kwargs = dict(
  metric="loss",
  mode="min",
)
tune_config = {
  "arch": "CNN",
  "input_len": 100,
  "output_dim": 1,
  "strand": tune.choice(["ss", "ds", "ts"]),
  "aggr": tune.choice(["max", "avg"]),
  "lr": tune.loguniform(1e-4, 1e-1),
  "batch_size": tune.choice([32, 64, 128]),
  "conv_kwargs": {
    "channels": [4, tune.choice([16, 32])],
    "conv_kernels": [tune.choice([3, 5])],
    "pool_kernels": [tune.choice([2, 4])],
    "dropout_rates": [tune.choice([0.1, 0.2])]
  },
  "fc_kwargs": {
    "hidden_dims": [32]
  }
}

In [25]:
res = hyperopt(
    tune_config,
    sdata=sdata,
    algorithm="HyperOptSearch",
    target_keys="activity_0",
    train_key="train_val",
    epochs=5,
    gpus=0,
    num_workers=0,
    algorithm_kwargs=algo_kwargs,
    scheduler_kwargs=scheduler_kwargs
)

Trial name,status,loc,aggr,batch_size,conv_kwargs/chann...,conv_kwargs/conv_...,conv_kwargs/dropo...,conv_kwargs/pool_...,lr,strand,iter,total time (s),loss
hyperopt_with_tune_8ddd6_00000,TERMINATED,172.16.5.4:17107,avg,32,16,3,0.1,2,0.0148132,ss,5,31.7328,0.0805183
hyperopt_with_tune_8ddd6_00001,TERMINATED,172.16.5.4:17274,avg,128,16,3,0.1,4,0.00038185,ts,5,20.8833,0.0821854
hyperopt_with_tune_8ddd6_00002,TERMINATED,172.16.5.4:17393,max,64,16,3,0.2,4,0.0243685,ss,5,11.0489,0.0841642
hyperopt_with_tune_8ddd6_00003,TERMINATED,172.16.5.4:17503,max,32,32,5,0.2,4,0.00449207,ss,5,3.34864,0.0822882
hyperopt_with_tune_8ddd6_00004,TERMINATED,172.16.5.4:17274,max,128,16,3,0.2,4,0.00120417,ss,5,1.67639,0.0826183
hyperopt_with_tune_8ddd6_00005,TERMINATED,172.16.5.4:17393,avg,128,32,5,0.2,2,0.0868617,ss,5,2.38077,0.655975
hyperopt_with_tune_8ddd6_00006,TERMINATED,172.16.5.4:17107,avg,32,16,5,0.1,4,0.000443137,ds,5,2.81773,0.0793859
hyperopt_with_tune_8ddd6_00007,TERMINATED,172.16.5.4:17274,avg,128,16,5,0.2,2,0.000842517,ds,5,1.45088,0.0805383
hyperopt_with_tune_8ddd6_00008,TERMINATED,172.16.5.4:17503,avg,128,16,5,0.2,4,0.07518,ss,5,1.27195,0.0875658
hyperopt_with_tune_8ddd6_00009,TERMINATED,172.16.5.4:17393,max,64,16,5,0.2,2,0.000779716,ts,5,2.31626,0.0840642


(pid=17107) Global seed set to 13
(hyperopt_with_tune pid=17107) Global seed set to 13
(hyperopt_with_tune pid=17107) /home/vscode/.local/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/callback_connector.py:91: LightningDeprecationWarning: Setting `Trainer(progress_bar_refresh_rate=0)` is deprecated in v1.5 and will be removed in v1.7. Please pass `pytorch_lightning.callbacks.progress.TQDMProgressBar` with `refresh_rate` directly to the Trainer's `callbacks` argument instead. Or, to disable the progress bar pass `enable_progress_bar = False` to the Trainer.
(hyperopt_with_tune pid=17107)   f"Setting `Trainer(progress_bar_refresh_rate={progress_bar_refresh_rate})` is deprecated in v1.5 and"
(hyperopt_with_tune pid=17107) GPU available: False, used: False
(hyperopt_with_tune pid=17107) TPU available: False, using: 0 TPU cores
(hyperopt_with_tune pid=17107) IPU available: False, using: 0 IPUs
(hyperopt_with_tune pid=17107) /home/vscode/.local/lib/python3.7/site-packages/

(hyperopt_with_tune pid=17107) Dropping 0 sequences with NaN targets.
(hyperopt_with_tune pid=17107) No transforms given, assuming just need to tensorize.
(hyperopt_with_tune pid=17107) No transforms given, assuming just need to tensorize.
(hyperopt_with_tune pid=17107) [<ray.tune.integration.pytorch_lightning.TuneReportCallback object at 0x7fa36f6b7690>]


(pid=17274) Global seed set to 13
(hyperopt_with_tune pid=17274) Global seed set to 13
(hyperopt_with_tune pid=17274) /home/vscode/.local/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/callback_connector.py:91: LightningDeprecationWarning: Setting `Trainer(progress_bar_refresh_rate=0)` is deprecated in v1.5 and will be removed in v1.7. Please pass `pytorch_lightning.callbacks.progress.TQDMProgressBar` with `refresh_rate` directly to the Trainer's `callbacks` argument instead. Or, to disable the progress bar pass `enable_progress_bar = False` to the Trainer.
(hyperopt_with_tune pid=17274)   f"Setting `Trainer(progress_bar_refresh_rate={progress_bar_refresh_rate})` is deprecated in v1.5 and"
(hyperopt_with_tune pid=17274) GPU available: False, used: False
(hyperopt_with_tune pid=17274) TPU available: False, using: 0 TPU cores
(hyperopt_with_tune pid=17274) IPU available: False, using: 0 IPUs
(hyperopt_with_tune pid=17274) /home/vscode/.local/lib/python3.7/site-packages/

(hyperopt_with_tune pid=17274) Dropping 0 sequences with NaN targets.
(hyperopt_with_tune pid=17274) No transforms given, assuming just need to tensorize.
(hyperopt_with_tune pid=17274) No transforms given, assuming just need to tensorize.
(hyperopt_with_tune pid=17274) [<ray.tune.integration.pytorch_lightning.TuneReportCallback object at 0x7fa7207e3f50>]


...

(hyperopt_with_tune pid=17503) Global seed set to 13
(hyperopt_with_tune pid=17503) GPU available: False, used: False
(hyperopt_with_tune pid=17503) TPU available: False, using: 0 TPU cores
(hyperopt_with_tune pid=17503) IPU available: False, using: 0 IPUs
(hyperopt_with_tune pid=17393) Global seed set to 13
(hyperopt_with_tune pid=17393) GPU available: False, used: False
(hyperopt_with_tune pid=17393) TPU available: False, using: 0 TPU cores
(hyperopt_with_tune pid=17393) IPU available: False, using: 0 IPUs
(hyperopt_with_tune pid=17503) 
(hyperopt_with_tune pid=17503)   | Name      | Type                      | Params
(hyperopt_with_tune pid=17503) --------------------------------------------------------
(hyperopt_with_tune pid=17503) 0 | hp_metric | R2Score                   | 0     
(hyperopt_with_tune pid=17503) 1 | convnet   | BasicConv1D               | 336   
(hyperopt_with_tune pid=17503) 2 | fcn       | BasicFullyConnectedModule | 12.4 K
(hyperopt_with_tune pid=17503) -------

(hyperopt_with_tune pid=17393) Dropping 0 sequences with NaN targets.
(hyperopt_with_tune pid=17393) No transforms given, assuming just need to tensorize.
(hyperopt_with_tune pid=17393) No transforms given, assuming just need to tensorize.
(hyperopt_with_tune pid=17393) [<ray.tune.integration.pytorch_lightning.TuneReportCallback object at 0x7fce8656b890>]


(hyperopt_with_tune pid=17393) Global seed set to 13


Result for hyperopt_with_tune_8ddd6_00008:
  date: 2022-11-02_19-43-44
  done: false
  experiment_id: 2976760426194f36a35d273bc8f3fbdb
  hostname: codespaces-97ce9f
  iterations_since_restore: 1
  loss: 0.5002864599227905
  node_ip: 172.16.5.4
  pid: 17503
  time_since_restore: 0.5047640800476074
  time_this_iter_s: 0.5047640800476074
  time_total_s: 0.5047640800476074
  timestamp: 1667418224
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: 8ddd6_00008
  warmup_time: 0.0038580894470214844
  
Result for hyperopt_with_tune_8ddd6_00007:
  date: 2022-11-02_19-43-44
  done: true
  experiment_id: ab0f8d65ad9942829324df5143d70a4c
  experiment_tag: 7_aggr=avg,batch_size=128,1=16,0=5,0=0.2000,0=2,lr=0.0008,strand=ds
  hostname: codespaces-97ce9f
  iterations_since_restore: 5
  loss: 0.08053827285766602
  node_ip: 172.16.5.4
  pid: 17274
  time_since_restore: 1.4508836269378662
  time_this_iter_s: 0.25186586380004883
  time_total_s: 1.4508836269378662
  timestamp: 1667418224
  ti

2022-11-02 19:43:46,937	INFO tune.py:759 -- Total run time: 48.20 seconds (47.90 seconds for the tuning loop).


In [26]:
res.best_config

{'arch': 'CNN',
 'input_len': 100,
 'output_dim': 1,
 'strand': 'ds',
 'aggr': 'avg',
 'lr': 0.00044313707981392264,
 'batch_size': 32,
 'conv_kwargs': {'channels': [4, 16],
  'conv_kernels': [5],
  'pool_kernels': [4],
  'dropout_rates': [0.1]},
 'fc_kwargs': {'hidden_dims': [32]}}

In [ ]:
trainable = tune.with_parameters(
    hyperopt_with_tune,
    sdata=sdata,
    target_keys="activity_0",
    train_key="train_val",
    epochs=10,
    gpus=0,
    num_workers=0
)

In [ ]:
tune_config = {
  "arch": "CNN",
  "input_len": 100,
  "output_dim": 1,
  "strand": tune.choice(["ss", "ds", "ts"]),
  "aggr": tune.choice(["max", "avg"]),
  "lr": tune.loguniform(1e-4, 1e-1),
  "batch_size": tune.choice([32, 64, 128]),
  "conv_kwargs": {
    "channels": [4, 16],
    "conv_kernels": [tune.choice([3, 5])],
    "pool_kernels": [tune.choice([2, 4])],
    "dropout_rates": [tune.choice([0.1, 0.2])]
  },
  "fc_kwargs": {
    "hidden_dims": [32]
  }
}

In [ ]:
scheduler = ASHAScheduler(
    metric="loss",
    mode="min",
    max_t=10,
    grace_period=1,
    reduction_factor=2
)

In [ ]:
analysis = tune.run(
    trainable,
    config=tune_config,
    metric="loss",
    mode="min",
    num_samples=10,
    local_dir=eu.settings.logging_dir,
    keep_checkpoints_num=1,
    checkpoint_score_attr="min-val_loss",
    name="test"
)

2022-11-01 20:14:11,580	WARNING services.py:1893 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 67104768 bytes available. This will harm performance! You may be able to free up space by deleting files in /dev/shm. If you are inside a Docker container, you can increase /dev/shm size by passing '--shm-size=1.97gb' to 'docker run' (or add it to the run_options list in a Ray cluster config). Make sure to set this to more than 30% of available RAM.
2022-11-01 20:14:12,966	INFO worker.py:1518 -- Started a local Ray instance.
/home/vscode/.local/lib/python3.7/site-packages/ray/tune/trainable/function_trainable.py:644: DeprecationWarning: `checkpoint_dir` in `func(config, checkpoint_dir)` is being deprecated. To save and load checkpoint in trainable functions, please use the `ray.air.session` API:

from ray.air import session

def train(config):
    # ...
    session.report({"metric": metric}, checkpoint=checkpoint)

For more information please see htt

Trial name,status,loc,aggr,batch_size,conv_kwargs/conv_...,conv_kwargs/dropo...,conv_kwargs/pool_...,lr,strand,iter,total time (s),loss
hyperopt_with_tune_c133d_00000,TERMINATED,172.16.5.4:6119,max,128,3,0.2,2,0.000515969,ts,10,27.7774,0.0812072
hyperopt_with_tune_c133d_00001,TERMINATED,172.16.5.4:6237,max,32,3,0.1,2,0.0022927,ss,10,19.5342,0.0842177
hyperopt_with_tune_c133d_00002,TERMINATED,172.16.5.4:6354,avg,64,5,0.1,2,0.0146573,ts,10,10.6593,0.0802305
hyperopt_with_tune_c133d_00003,TERMINATED,172.16.5.4:6459,avg,128,5,0.2,4,0.0372549,ss,10,1.73107,0.0891374
hyperopt_with_tune_c133d_00004,TERMINATED,172.16.5.4:6459,max,32,3,0.1,2,0.0954421,ts,10,5.44144,0.0802142
hyperopt_with_tune_c133d_00005,TERMINATED,172.16.5.4:6119,max,64,3,0.1,4,0.0656628,ts,10,3.69808,0.0802464
hyperopt_with_tune_c133d_00006,TERMINATED,172.16.5.4:6354,avg,64,5,0.2,4,0.00775331,ss,10,3.4115,0.085105
hyperopt_with_tune_c133d_00007,TERMINATED,172.16.5.4:6237,max,128,5,0.2,4,0.000389489,ds,10,2.96298,0.082369
hyperopt_with_tune_c133d_00008,TERMINATED,172.16.5.4:6354,max,128,3,0.2,2,0.0735737,ts,10,1.9596,0.0837441
hyperopt_with_tune_c133d_00009,TERMINATED,172.16.5.4:6119,avg,128,5,0.1,4,0.000654435,ts,10,1.9034,0.079416


(pid=6119) Global seed set to 13
(hyperopt_with_tune pid=6119) Global seed set to 13
(hyperopt_with_tune pid=6119) /home/vscode/.local/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/callback_connector.py:91: LightningDeprecationWarning: Setting `Trainer(progress_bar_refresh_rate=0)` is deprecated in v1.5 and will be removed in v1.7. Please pass `pytorch_lightning.callbacks.progress.TQDMProgressBar` with `refresh_rate` directly to the Trainer's `callbacks` argument instead. Or, to disable the progress bar pass `enable_progress_bar = False` to the Trainer.
(hyperopt_with_tune pid=6119)   f"Setting `Trainer(progress_bar_refresh_rate={progress_bar_refresh_rate})` is deprecated in v1.5 and"
(hyperopt_with_tune pid=6119) GPU available: False, used: False
(hyperopt_with_tune pid=6119) TPU available: False, using: 0 TPU cores
(hyperopt_with_tune pid=6119) IPU available: False, using: 0 IPUs
(hyperopt_with_tune pid=6119) /home/vscode/.local/lib/python3.7/site-packages/pytorch_

(hyperopt_with_tune pid=6119) Dropping 0 sequences with NaN targets.
(hyperopt_with_tune pid=6119) No transforms given, assuming just need to tensorize.
(hyperopt_with_tune pid=6119) No transforms given, assuming just need to tensorize.
(hyperopt_with_tune pid=6119) [<ray.tune.integration.pytorch_lightning.TuneReportCallback object at 0x7f3984575a10>]


(hyperopt_with_tune pid=6119) Global seed set to 13
(hyperopt_with_tune pid=6119) /home/vscode/.local/lib/python3.7/site-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
(hyperopt_with_tune pid=6119)   f"The dataloader, {name}, does not have many workers which may be a bottleneck."
(hyperopt_with_tune pid=6119) /home/vscode/.local/lib/python3.7/site-packages/pytorch_lightning/trainer/data_loading.py:433: UserWarning: The number of training samples (7) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
(hyperopt_with_tune pid=6119)   f"The number of training samples ({self.num_training_batches}) is smaller than the log

...

(hyperopt_with_tune pid=6354) Global seed set to 13
(hyperopt_with_tune pid=6354) GPU available: False, used: False
(hyperopt_with_tune pid=6354) TPU available: False, using: 0 TPU cores
(hyperopt_with_tune pid=6354) IPU available: False, using: 0 IPUs
(hyperopt_with_tune pid=6119) Global seed set to 13
(hyperopt_with_tune pid=6119) GPU available: False, used: False
(hyperopt_with_tune pid=6119) TPU available: False, using: 0 TPU cores
(hyperopt_with_tune pid=6119) IPU available: False, using: 0 IPUs
(hyperopt_with_tune pid=6119) 
(hyperopt_with_tune pid=6119)   | Name            | Type                      | Params
(hyperopt_with_tune pid=6119) --------------------------------------------------------------
(hyperopt_with_tune pid=6119) 0 | hp_metric       | R2Score                   | 0     
(hyperopt_with_tune pid=6119) 1 | convnet         | BasicConv1D               | 336   
(hyperopt_with_tune pid=6119) 2 | reverse_convnet | BasicConv1D               | 336   
(hyperopt_with_tune pi

Result for hyperopt_with_tune_c133d_00007:
  date: 2022-11-01_20-14-55
  done: true
  experiment_id: 5cc423fa139b475bb55725e4c27d3c03
  experiment_tag: 7_aggr=max,batch_size=128,0=5,0=0.2000,0=4,lr=0.0004,strand=ds
  hostname: codespaces-97ce9f
  iterations_since_restore: 10
  loss: 0.08236898481845856
  node_ip: 172.16.5.4
  pid: 6237
  time_since_restore: 2.9629783630371094
  time_this_iter_s: 0.21939635276794434
  time_total_s: 2.9629783630371094
  timestamp: 1667333695
  timesteps_since_restore: 0
  training_iteration: 10
  trial_id: c133d_00007
  warmup_time: 0.0030231475830078125
  
Result for hyperopt_with_tune_c133d_00009:
  date: 2022-11-01_20-14-56
  done: false
  experiment_id: a96250e996b1433ebcf48bb5b6a3d41b
  hostname: codespaces-97ce9f
  iterations_since_restore: 1
  loss: 0.10714306682348251
  node_ip: 172.16.5.4
  pid: 6119
  time_since_restore: 0.2818934917449951
  time_this_iter_s: 0.2818934917449951
  time_total_s: 0.2818934917449951
  timestamp: 1667333696
  timest

(hyperopt_with_tune pid=6119) Global seed set to 13


Result for hyperopt_with_tune_c133d_00004:
  date: 2022-11-01_20-14-57
  done: true
  experiment_id: 0e2662855e564568801e80eac2f05e4a
  experiment_tag: 4_aggr=max,batch_size=32,0=3,0=0.1000,0=2,lr=0.0954,strand=ts
  hostname: codespaces-97ce9f
  iterations_since_restore: 10
  loss: 0.08021415770053864
  node_ip: 172.16.5.4
  pid: 6459
  time_since_restore: 5.441444158554077
  time_this_iter_s: 0.3250539302825928
  time_total_s: 5.441444158554077
  timestamp: 1667333697
  timesteps_since_restore: 0
  training_iteration: 10
  trial_id: c133d_00004
  warmup_time: 0.0030121803283691406
  
Result for hyperopt_with_tune_c133d_00009:
  date: 2022-11-01_20-14-57
  done: true
  experiment_id: a96250e996b1433ebcf48bb5b6a3d41b
  experiment_tag: 9_aggr=avg,batch_size=128,0=5,0=0.1000,0=4,lr=0.0007,strand=ts
  hostname: codespaces-97ce9f
  iterations_since_restore: 10
  loss: 0.07941604405641556
  node_ip: 172.16.5.4
  pid: 6119
  time_since_restore: 1.903404712677002
  time_this_iter_s: 0.15060281

2022-11-01 20:14:57,962	INFO tune.py:759 -- Total run time: 43.97 seconds (43.63 seconds for the tuning loop).


In [ ]:
analysis.results_df

,loss,time_this_iter_s,done,timesteps_total,episodes_total,training_iteration,experiment_id,date,timestamp,time_total_s,...,config/output_dim,config/strand,config/aggr,config/lr,config/batch_size,config/conv_kwargs/channels,config/conv_kwargs/conv_kernels,config/conv_kwargs/pool_kernels,config/conv_kwargs/dropout_rates,config/fc_kwargs/hidden_dims
trial_id,,,,,,,,,,,,,,,,,,,,,
c133d_00000,0.081207,0.308473,True,None,None,10,a96250e996b1433ebcf48bb5b6a3d41b,2022-11-01_20-14-51,1667333691,27.777361,...,1,ts,max,0.000516,128,"[4, 16]",[3],[2],[0.2],[32]
c133d_00001,0.084218,0.346641,True,None,None,10,5cc423fa139b475bb55725e4c27d3c03,2022-11-01_20-14-52,1667333692,19.534242,...,1,ss,max,0.002293,32,"[4, 16]",[3],[2],[0.1],[32]
c133d_00002,0.080231,0.345087,True,None,None,10,2033aa1d5f7047aaabba2e6f654f3661,2022-11-01_20-14-52,1667333692,10.659317,...,1,ts,avg,0.014657,64,"[4, 16]",[5],[2],[0.1],[32]
c133d_00003,0.089137,0.111302,True,None,None,10,0e2662855e564568801e80eac2f05e4a,2022-11-01_20-14-51,1667333691,1.731071,...,1,ss,avg,0.037255,128,"[4, 16]",[5],[4],[0.2],[32]
c133d_00004,0.080214,0.325054,True,None,None,10,0e2662855e564568801e80eac2f05e4a,2022-11-01_20-14-57,1667333697,5.441444,...,1,ts,max,0.095442,32,"[4, 16]",[3],[2],[0.1],[32]
c133d_00005,0.080246,0.388377,True,None,None,10,a96250e996b1433ebcf48bb5b6a3d41b,2022-11-01_20-14-55,1667333695,3.698084,...,1,ts,max,0.065663,64,"[4, 16]",[3],[4],[0.1],[32]
c133d_00006,0.085105,0.247247,True,None,None,10,2033aa1d5f7047aaabba2e6f654f3661,2022-11-01_20-14-55,1667333695,3.411499,...,1,ss,avg,0.007753,64,"[4, 16]",[5],[4],[0.2],[32]
c133d_00007,0.082369,0.219396,True,None,None,10,5cc423fa139b475bb55725e4c27d3c03,2022-11-01_20-14-55,1667333695,2.962978,...,1,ds,max,0.000389,128,"[4, 16]",[5],[4],[0.2],[32]
c133d_00008,0.083744,0.119390,True,None,None,10,2033aa1d5f7047aaabba2e6f654f3661,2022-11-01_20-14-57,1667333697,1.959604,...,1,ts,max,0.073574,128,"[4, 16]",[3],[2],[0.2],[32]


In [4]:
config = {
  "arch": "CNN",
  "input_len": 100,
  "output_dim": 1,
  "strand": "ss",
  "aggr": None,
  "lr": 1e-3,
  "batch_size": 64,
  "conv_kwargs": {
    "channels": [4, 16],
    "conv_kernels": [3],
    "pool_kernels": [2],
    "dropout_rates": [0.1]
  },
  "fc_kwargs": {
    "hidden_dims": [32]
  }
}

In [5]:
model = eu.models.get_model(config["arch"], config)
model

CNN(
  (hp_metric): R2Score()
  (convnet): BasicConv1D(
    (module): Sequential(
      (0): Conv1d(4, 16, kernel_size=(3,), stride=(1,), padding=valid)
      (1): ReLU()
      (2): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (3): Dropout(p=0.1, inplace=False)
    )
  )
  (fcn): BasicFullyConnectedModule(
    (module): Sequential(
      (0): Linear(in_features=784, out_features=32, bias=True)
      (1): ReLU()
      (2): Linear(in_features=32, out_features=1, bias=True)
    )
  )
)

In [16]:
eu.train.fit(
    model = model,
    sdata = sdata,
    target_keys = "activity_0", 
    train_key = "train_val",
    epochs = 10,
    gpus = 0,
    num_workers = 0
)

Global seed set to 13
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs

  | Name      | Type                      | Params
--------------------------------------------------------
0 | hp_metric | R2Score                   | 0     
1 | convnet   | BasicConv1D               | 208   
2 | fcn       | BasicFullyConnectedModule | 25.2 K
--------------------------------------------------------
25.4 K    Trainable params
0         Non-trainable params
25.4 K    Total params
0.101     Total estimated model params size (MB)


Dropping 0 sequences with NaN targets.
No transforms given, assuming just need to tensorize.
No transforms given, assuming just need to tensorize.


Validation sanity check: 0it [00:00, ?it/s]

/home/vscode/.local/lib/python3.7/site-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."
Global seed set to 13
/home/vscode/.local/lib/python3.7/site-packages/pytorch_lightning/trainer/data_loading.py:133: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 4 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  f"The dataloader, {name}, does not have many workers which may be a bottleneck."
/home/vscode/.local/lib/python3.7/site-packages/pytorch_lightning/trainer/data_loading.py:433: U

Training: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [11]:
analysis.results_df.columns

Index(['loss', 'time_this_iter_s', 'done', 'timesteps_total', 'episodes_total',
       'training_iteration', 'experiment_id', 'date', 'timestamp',
       'time_total_s', 'pid', 'hostname', 'node_ip', 'time_since_restore',
       'timesteps_since_restore', 'iterations_since_restore', 'warmup_time',
       'experiment_tag', 'config/arch', 'config/input_len',
       'config/output_dim', 'config/strand', 'config/aggr', 'config/lr',
       'config/batch_size', 'config/conv_kwargs/channels',
       'config/conv_kwargs/conv_kernels', 'config/conv_kwargs/pool_kernels',
       'config/conv_kwargs/dropout_rates', 'config/fc_kwargs/hidden_dims'],
      dtype='object')

---

In [9]:
def train_tune(config, train_dataset, val_dataset, epochs=10, gpus=0):
  model =  eu.models.FCN(
    input_len=100, 
    output_dim=1, 
    lr=config["lr"]
    )
  train_dl = train_dataset.to_dataloader(batch_size=config["batch_size"])
  val_dl = val_dataset.to_dataloader(batch_size=config["batch_size"])
  trainer = Trainer(
    max_epochs=epochs,
    gpus=gpus,
    progress_bar_refresh_rate=0,
    callbacks=[callback])
  trainer.fit(model, train_dataloaders=train_dl, val_dataloaders=val_dl)

In [10]:
trainable = tune.with_parameters(
    train_tune,
    train_dataset=sdataset_train,
    val_dataset=sdataset_val,
    epochs=10,
    gpus=0
)

In [48]:
best_trial = analysis.best_trial  # Get best trial
best_config = analysis.best_config  # Get best trial's hyperparameters
best_logdir = analysis.best_logdir  # Get best trial's logdir
best_checkpoint = analysis.best_checkpoint  # Get best trial's best checkpoint
best_result = analysis.best_result  # Get best trial's last results

2022-10-19 03:34:19,999	ERROR experiment_analysis.py:486 -- No checkpoints have been found for trial train_tune_86a07_00000.


In [49]:
best_trial

train_tune_86a07_00000

In [50]:
best_config

{'lr': 0.00010000831284081109, 'batch_size': 32}

In [51]:
best_logdir

'/home/vscode/ray_results/test/train_tune_86a07_00000_0_batch_size=32,lr=0.0001_2022-10-19_03-31-32'

In [52]:
best_checkpoint

In [53]:
best_result

{'loss': 0.11025925725698471,
 'time_this_iter_s': 0.11937999725341797,
 'done': True,
 'timesteps_total': None,
 'episodes_total': None,
 'training_iteration': 10,
 'trial_id': '86a07_00000',
 'experiment_id': '299e91b7003d4457b77be9e3b61a2294',
 'date': '2022-10-19_03-31-51',
 'timestamp': 1666150311,
 'time_total_s': 10.0547194480896,
 'pid': 23527,
 'hostname': 'codespaces-97ce9f',
 'node_ip': '172.16.5.4',
 'config': {'lr': 0.00010000831284081109, 'batch_size': 32},
 'time_since_restore': 10.0547194480896,
 'timesteps_since_restore': 0,
 'iterations_since_restore': 10,
 'warmup_time': 0.0036079883575439453,
 'experiment_tag': '0_batch_size=32,lr=0.0001'}

Bad pipe message: %s [b'Z\xe4~\xb8\x0c\\\x08\x1b`', b'\x93\xa5x\xa8\xf9\x18\xaf \xc7\x85\x1f\xd3\x93x\x86k9\xcd\xfe C\xf5\x80\xd7\xd2\x85\xaeE\x0e_\xbeHF\xe6\xc2p\xe0`W\xab\x00\x08\x13\x02\x13\x03\x13\x01\x00\xff\x01\x00\x00', b'\x00\x00\x0e\x00\x0c\x00\x00\t127.0.0.1\x00\x0b\x00\x04\x03\x00\x01\x02\x00\n\x00\x0c\x00\n\x00\x1d\x00\x17\x00\x1e\x00\x19\x00\x18\x00#\x00\x00\x00\x16\x00\x00\x00\x17\x00\x00\x00\r\x00\x1e\x00\x1c\x04\x03\x05\x03\x06\x03\x08\x07\x08\x08\x08\t\x08\n\x08\x0b\x08\x04\x08\x05\x08\x06\x04\x01\x05\x01\x06\x01\x00+\x00\x03\x02\x03\x04\x00-\x00\x02\x01\x01\x003\x00&\x00$\x00\x1d\x00 /\x934\x9f\xd5\x8eo7\x8d\xe5rv\xbc1z^B\x17%\x86\x00\x9cn\x82\xebF\xf6*Qy\t\x1e']
Bad pipe message: %s [b"q8\xf8\xce\xd9O\xe02\xaeh\x96\xe3\ru'\xa7\xff\xba O\xe0\xf9\x88\xe3\x04{\x1em\xf6\xe3Zt\xb5\x8e\xa3-\xd1\xba=\xd1`-\xfdC\xca\xde$\x1a\x85V\xc6\x00\x08\x13\x02\x13"]
Bad pipe message: %s [b'\x12\xae\xc8\xf1gV\r\x002\xe63\x8d\x9e\x1b\x14>O\xcc\x00\x00|\xc0,\xc00\x00\xa3\x00\x9f\xcc\xa9\x